In [ ]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key:")

Enter Your OpenAI API Key:··········


In [ ]:
pip install langchain

In [ ]:
from langchain.globals import set_verbose

set_verbose(True)

#🔗 **Origins of CoT Prompting**

- CoT was introduced by Wei et al. in their 2022 paper, [Chain-of-Thought Prompting Elicits Reasoning in Large Language Models](https://browse.arxiv.org/pdf/2201.11903.pdf).

- It's a response to the need for better reasoning in large language models.

💡 **What is CoT Prompting?**
- CoT stands for Chain of Thought Prompting.

- It's about guiding language models through a step-by-step reasoning process.

- The model shows its reasoning, forming a "chain" of thoughts, rather than just giving an answer.

- Mimics human-like thought processes for complex tasks.


### 🎓 **Standard vs. CoT Prompting Example**

<figure>
  <img src="https://deepgram.com/_next/image?url=https%3A%2F%2Fwww.datocms-assets.com%2F96965%2F1696369825-cot-example.png&w=3840&q=75" alt="Image Description" style="width:100%">
  <figcaption>Examples of ⟨input, chain of thought, output⟩ triples for arithmetic, commonsense, and symbolic reasoning benchmarks. Chains of thought are highlighted.</figcaption>
  <a href="https://deepgram.com/_next/image?url=https%3A%2F%2Fwww.datocms-assets.com%2F96965%2F1696369825-cot-example.png&w=3840&q=75">Image Source</a>
</figure>

- Standard: Direct answer with no reasoning (e.g., "11").

- CoT: Detailed reasoning steps (e.g., "Roger started with 5, added 6 from 2 cans, so 5 + 6 = 11").

### 🚀 **Usage of CoT Prompting**

1. **Enhanced Reasoning**: Breaks down complex problems into simpler steps.

2. **Combination with Few-Shot Prompting**: Uses examples to guide model responses, great for complex tasks.

3. **Interpretability**: Offers a clear view of the model's thought process.

4. **Applications**: Useful in arithmetic, commonsense reasoning, and symbolic tasks.

### ✨ **Impact of CoT Prompting**

- Improves accuracy and insightfulness in responses.

- Focuses on reasoning and interpretability, especially in complex scenarios.

In [ ]:
pip install datasets

The code below downloads an dataset which has over [1.8 million Chain of Thought examples](https://huggingface.co/datasets/kaist-ai/CoT-Collection).

In [ ]:
from datasets import load_dataset

dataset = load_dataset("kaist-ai/CoT-Collection", split="train", trust_remote_code=True)

In [ ]:
dataset = dataset.remove_columns(['task', 'type'])

dataset = dataset.shuffle(seed=42)

dataset = dataset.select(range(10_000))

dataset = dataset.to_pandas()

dataset.head()

,source,target,rationale
0,Question: What about Neptune did NASA propose ...,no,The proposed mission to Neptune is not mention...
1,Choose your reply from the options at the end....,yes,The passage talks about the Tucson-Pima County...
2,You are given an original reference as well as...,1,The system generated reference is grammaticall...
3,"Is the premise ""A person on a beach in a green...",yes,The premise describes a situation where there ...
4,Pick the option in line with common sense to a...,C,"The rationale is: ""C, expected because it was ..."


We'll sample just 10,000 of these Chain of Thought examples and save them as a list of dictionaries.

In [ ]:
selected_examples = dataset.to_dict(orient='records')

In [ ]:
selected_examples[0]

{'source': 'Question: What about Neptune did NASA propose in 2003 in their "Vision Missions Studies"?\n\nIs However, there have been a couple of discussions to launch Neptune missions sooner. a good answer to this question?\n\nOPTIONS:\n- yes\n- no',
 'target': 'no',
 'rationale': 'The proposed mission to Neptune is not mentioned. The only mention of the planet in this excerpt is a note that there have been proposals for such missions, but they are very distant and probably will never happen.'}

For this example we'll use an open source embedding model from HuggingFace so we don't accumulate a huge bill for OpenAI

In [ ]:
pip install langchain-community sentence-transformers

In [ ]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-base-en-v1.5"

encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cuda'},
    encode_kwargs=encode_kwargs
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

To set up a Chain of Thought prompt you will use a `FewShotPromptTemplate` as well as an example selector.

In this example you'll use `MaxMarginalRelevanceExampleSelector`, but you can use any of the example selectors you learned about before.

In [ ]:
from langchain.prompts.example_selector import MaxMarginalRelevanceExampleSelector

from langchain.prompts import FewShotPromptTemplate, PromptTemplate

from langchain_community.vectorstores import FAISS

In [ ]:
prefix = "Consider the following as examples of how to reason:"

examples_template = """Query: {source}

Rationale: {rationale}

Response: {target}
"""

suffix = """Using a similar reasoning approach, answer the users question which is delimited by triple backticks.

User question: ```{input}```

Take a deep breath, break down the user's query step-by-step, and provide a clear chain of thought in your response."
"""

In [ ]:
examples_prompt = PromptTemplate(
    input_variables=["source", "rationale", "target"],
    template=examples_template
)

There's a large number of examples to embed and add to the vector store. The below cell will take a few minutes to run. It took me ~3 minutes using the free T4 GPU provided on Google colab.

In [ ]:
pip install faiss-gpu

In [ ]:
example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
    # The list of examples available to select from.
    selected_examples,
    # The embedding class used to produce embeddings which are used to measure semantic similarity.
    embeddings,
    # The VectorStore class that is used to store the embeddings and do a similarity search over.
    FAISS,
    # The number of examples to produce.
    k=7,
)

In [ ]:
mmr_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=examples_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input"]
)

In [ ]:
query = """Here's a short story: A lifter uses a barbell, but moves a jump rope\
in a wider arc. The object likely to travel further is (A) the jump rope (B) the\
barbell.

What is the most sensical answer between "a jump rope" and "a barbell"?
"""

In [ ]:
print(mmr_prompt.format(input=query))

Consider the following as examples of how to reason:

Query: In this task, you will be shown a short story with a beginning, two potential middles, and an ending. Your job is to choose the middle statement that makes the story coherent / plausible by writing "1" or "2" in the output. If both sentences are plausible, pick the one that makes most sense.

Beginning: I was in gymnastics class one day. Middle 1: I had stumbled on a vaulting horse and injured. Middle 2: I  had my best performer on a horse. Ending: This event ended my gymnastics career.

Rationale: Middle 1 indicates that I had stumbled on a vaulting horse, which resulted in an injury. This event ended my gymnastics career. Middle 2 does not make sense because a gymnast's best performance would not be on the vaulting horse, but rather on other apparatuses such as the balance beam or uneven bars.

Response: 1


Query: I am testing my students' logic.
What is the answer they should choose between "Steel cable" and "Rubber cable

In [ ]:
pip install langchain-openai

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model="gpt-4-0125-preview", temperature=0.0)

llm_chain = mmr_prompt | llm | StrOutputParser()

for chunk in llm_chain.stream({"input":query}):
  print(chunk, end="", flush=True)

The user's query involves understanding the dynamics of motion for two different objects: a jump rope and a barbell, when used by a lifer. The key point to consider here is the manner in which each object is moved - the jump rope is moved in a wider arc, while the barbell is simply lifted.

When considering the potential for an object to "travel further," we're essentially looking at the range of motion and the ability of the object to cover distance within its path of movement. A jump rope, when swung in a wider arc, has the potential to cover a larger area due to its flexibility and the nature of its use. It's designed to be swung around the body, and doing so in a wider arc increases its range. On the other hand, a barbell is primarily used for lifting and is constrained by the lifter's strength and the gravitational pull. Its movement is mostly vertical, and it does not "travel" in the sense of covering ground horizontally.

Given these considerations, the object likely to "travel 

In [ ]:
query = """
Given the fact that: Inhaling, or breathing in, increases the size of the chest, \
 which decreases air pressure inside the lungs. Answer the question: If Mona is \
 done with a race and her chest contracts, what happens to the amount of air \
 pressure in her lungs increases or decreases?
"""

In [ ]:
for chunk in llm_chain.stream({"input":query}):
  print(chunk, end="", flush=True)

Given the fact that inhaling, or breathing in, increases the size of the chest, which decreases air pressure inside the lungs, we can infer the opposite action—when the chest contracts (which occurs during exhaling)—would lead to an increase in air pressure inside the lungs. This is because the volume of the chest cavity decreases, causing the air within the lungs to be compressed, which increases the pressure. 

So, if Mona is done with a race and her chest contracts, the amount of air pressure in her lungs increases. 

Rationale: The principle that guides this reasoning is Boyle's law, which states that pressure and volume have an inverse relationship when temperature is held constant. When Mona's chest contracts after a race, the volume of her chest cavity decreases, leading to an increase in the pressure of the air inside her lungs. 

Response: increases